<a href="https://colab.research.google.com/github/isabellacadisco/Algorithms-for-Massive-Data-course/blob/main/LSH6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
#set up os env
os.environ['KAGGLE_USERNAME'] = "isabellacadisco"
os.environ['KAGGLE_KEY'] = "847325691c2399e41dc9c0b78fbe499e"

# download dataset from kaggle and save it in drive
!kaggle datasets download -d xhlulu/medal-emnlp

100% 6.82G/6.82G [01:26<00:00, 48.3MB/s]
100% 6.82G/6.82G [01:26<00:00, 84.8MB/s]


In [2]:
!unzip medal-emnlp.zip #unzip dataset

Archive:  medal-emnlp.zip
  inflating: full_data.csv           
  inflating: pretrain_subset/test.csv  
  inflating: pretrain_subset/train.csv  
  inflating: pretrain_subset/valid.csv  


In [4]:
# set up spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null #install java as VM
!wget -q https://dlcdn.apache.org/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz #download Spark
!tar xf spark-3.4.0-bin-hadoop3.tgz #unzip the file
!pip install -q findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.0-bin-hadoop3"

import findspark
findspark.init("spark-3.4.0-bin-hadoop3")# SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate() #SPARK SESSION

import pyspark
type(spark)

pyspark.sql.session.SparkSession

In [5]:
file_path = './full_data.csv'

In [6]:
data_df = spark.read.csv(file_path, header=True)

In [7]:
from pyspark.sql.functions import monotonically_increasing_id

In [8]:
dataset = data_df.select('TEXT').withColumn('doc_id', monotonically_increasing_id())

In [9]:
dataset.show()

+--------------------+------+
|                TEXT|doc_id|
+--------------------+------+
|alphabisabolol ha...|     0|
|a report is given...|     1|
|the virostatic co...|     2|
|rmi rmi and rmi a...|     3|
|a doubleblind stu...|     4|
|stroma from eithe...|     5|
|the effect of the...|     6|
|in one experiment...|     7|
|the presence of a...|     8|
|the reaction of g...|     9|
|choline acetyltra...|    10|
|increasing concen...|    11|
|the properties of...|    12|
|primary amines re...|    13|
|a purification pr...|    14|
|dihydrofolate red...|    15|
|ionization effect...|    16|
|kinetic analyses ...|    17|
|the nearultraviol...|    18|
|the circular pola...|    19|
+--------------------+------+
only showing top 20 rows



In [10]:
small_dataset = dataset.sample(withReplacement=False, fraction=0.00007, seed=42)

In [11]:
small_dataset.count()

935

In [12]:
small_dataset.show()

+--------------------+----------+
|                TEXT|    doc_id|
+--------------------+----------+
|guinea worm disea...|     54857|
|clinically uninvo...|     60617|
|the clinical stru...|     66206|
|in a lot of patie...|     71617|
|human term placen...|     72115|
|a recent outbreak...|    106539|
|qualitative and q...|    117032|
|four patients wit...|    117862|
|as life expectanc...|    125607|
|the purpose of th...|    125621|
|gastrointestinal ...|    129527|
|previous experime...|8589974423|
|mice received rad...|8589977320|
|food intake and r...|8589989283|
|the phenomenon of...|8590016766|
|the possibility o...|8590033288|
|a method is descr...|8590041248|
|the effect of int...|8590045842|
|muscle biopsy tis...|8590063333|
|approximately pre...|8590069516|
+--------------------+----------+
only showing top 20 rows



# Text preprocessing

In [13]:
# TEXT PROCESSOR
!pip install spacy
!python -m spacy download en_core_web_sm

# Import spaCy and download the English model
import spacy
import en_core_web_sm

# Load the spaCy English model FORSE NON SERVE BOH
nlp = en_core_web_sm.load()

import string

def preprocess_text(text):
    # Initialize spaCy resources
    nlp = spacy.load("en_core_web_sm")

    # Apply spaCy NLP pipeline
    doc = nlp(text)

    # Lemmatize and remove stopwords
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and token.text not in string.punctuation]

    # Join tokens back into a text
    cleaned_text = " ".join(tokens)

    return cleaned_text

2023-09-13 16:20:49.336031: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-13 16:20:51.415642: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 31.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [14]:
# UDF --

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Register the UDF with Spark
preprocess_udf_spark = udf(preprocess_text, StringType())

In [15]:
df_prep = small_dataset.withColumn("preprocessed_text", preprocess_udf_spark(small_dataset['TEXT']))

In [16]:
df_prep.show()

+--------------------+----------+--------------------+
|                TEXT|    doc_id|   preprocessed_text|
+--------------------+----------+--------------------+
|guinea worm disea...|     54857|guinea worm disea...|
|clinically uninvo...|     60617|clinically uninvo...|
|the clinical stru...|     66206|clinical structur...|
|in a lot of patie...|     71617|lot patient form ...|
|human term placen...|     72115|human term placen...|
|a recent outbreak...|    106539|recent outbreak s...|
|qualitative and q...|    117032|qualitative quant...|
|four patients wit...|    117862|patient pulmonary...|
|as life expectanc...|    125607|life expectancy i...|
|the purpose of th...|    125621|purpose t0 determ...|
|gastrointestinal ...|    129527|gastrointestinal ...|
|previous experime...|8589974423|previous experime...|
|mice received rad...|8589977320|mouse receive rad...|
|food intake and r...|8589989283|food intake rate ...|
|the phenomenon of...|8590016766|phenomenon photoa...|
|the possi

In [17]:
df = df_prep.select('doc_id','preprocessed_text')

# Len of docs in chars

In [18]:
from pyspark.sql.functions import length


In [19]:
# Add a new column containing the character count for each row
df_with_char_count = df.withColumn("CharacterCount", length(df["preprocessed_text"]))

# Show the resulting DataFrame
df_with_char_count.show()

+----------+--------------------+--------------+
|    doc_id|   preprocessed_text|CharacterCount|
+----------+--------------------+--------------+
|     54857|guinea worm disea...|           459|
|     60617|clinically uninvo...|           487|
|     66206|clinical structur...|           713|
|     71617|lot patient form ...|           590|
|     72115|human term placen...|           927|
|    106539|recent outbreak s...|           518|
|    117032|qualitative quant...|          1228|
|    117862|patient pulmonary...|           450|
|    125607|life expectancy i...|           480|
|    125621|purpose t0 determ...|           435|
|    129527|gastrointestinal ...|           696|
|8589974423|previous experime...|          1604|
|8589977320|mouse receive rad...|          1073|
|8589989283|food intake rate ...|           666|
|8590016766|phenomenon photoa...|           614|
|8590033288|possibility patie...|           638|
|8590041248|method describe r...|           442|
|8590045842|effect i

k = 5, 5 bytes
hashed to 2^16, 2 bytes

In [20]:
# HASH FAMILY


# ci sono degli import che non ho eseguito ma ho potuto comunque creare classe
# forse sono già in colab boh

import re, hashlib, math, time
from random import randint, seed

# però sono abbastanza certa che il seed mi serva

seed(16)

# HO MESSO BYTE RITORNO A 4, PENSO 2^32 BUCKET THEN
# BUT WHO KNOWS

class hashFamily:
    def __init__(self, i):
        self.resultSize = 2 # how many bytes we want back
        self.maxLen = 20 # how long can our i be (in decimal)
        self.salt = str(i).zfill(self.maxLen)[-self.maxLen:]
        self.id = i

    def get_hash_value(self, el_to_hash):
        return int(hashlib.sha1(str(el_to_hash).encode('utf-8') + self.salt.encode('utf-8')).hexdigest()[-self.resultSize:], 16)


In [21]:
# SHINGLER


class shingler:
    def __init__(self, k):

        if k > 0:
            self.k = int(k)
        else:
            self.k = 5

    #inner class utility
    #def process_doc(self, document):
     #   return re.sub("( )+|(\n)+"," ",document).lower()

    def get_shingles(self, document):
        shingles = set()
        for i in range(0, len(document)-self.k+1 ):
            shingles.add(document[i:i+self.k])
        return shingles

    def get_k(self):
        return self.k

    #return sorted hash
    def get_hashed_shingles(self, shingles_set):
        hash_function = hashFamily(0)
        return sorted( {hash_function.get_hash_value(s) for s in shingles_set} )

In [22]:
def shingling_map(row):
    out = []
    sh_instance = shingler(8)
    hashed_shingles = sh_instance.get_hashed_shingles( sh_instance.get_shingles( row['preprocessed_text'] ) )
    signature_size = 500
    for i in range(0,signature_size): #signature size
        out.append( (row['doc_id'], hashed_shingles, i ) )
    #return an iterator to use flatMap => produce more than one key-value pair as output (namely one per hash function)
    return iter(out)

In [23]:
docId_shingleset_hfunc = df.rdd.flatMap(shingling_map)

In [24]:
docId_shingleset_hfunc.toDF().show()

+-----+--------------------+---+
|   _1|                  _2| _3|
+-----+--------------------+---+
|54857|[0, 1, 2, 3, 4, 5...|  0|
|54857|[0, 1, 2, 3, 4, 5...|  1|
|54857|[0, 1, 2, 3, 4, 5...|  2|
|54857|[0, 1, 2, 3, 4, 5...|  3|
|54857|[0, 1, 2, 3, 4, 5...|  4|
|54857|[0, 1, 2, 3, 4, 5...|  5|
|54857|[0, 1, 2, 3, 4, 5...|  6|
|54857|[0, 1, 2, 3, 4, 5...|  7|
|54857|[0, 1, 2, 3, 4, 5...|  8|
|54857|[0, 1, 2, 3, 4, 5...|  9|
|54857|[0, 1, 2, 3, 4, 5...| 10|
|54857|[0, 1, 2, 3, 4, 5...| 11|
|54857|[0, 1, 2, 3, 4, 5...| 12|
|54857|[0, 1, 2, 3, 4, 5...| 13|
|54857|[0, 1, 2, 3, 4, 5...| 14|
|54857|[0, 1, 2, 3, 4, 5...| 15|
|54857|[0, 1, 2, 3, 4, 5...| 16|
|54857|[0, 1, 2, 3, 4, 5...| 17|
|54857|[0, 1, 2, 3, 4, 5...| 18|
|54857|[0, 1, 2, 3, 4, 5...| 19|
+-----+--------------------+---+
only showing top 20 rows



In [25]:
docId_shingleset_hfunc.count()

467500

In [26]:
def minhash_map(docId_ShingleSet_hFunct):
    doc_id = docId_ShingleSet_hFunct[0]
    shingles = docId_ShingleSet_hFunct[1]
    hash_f = hashFamily( docId_ShingleSet_hFunct[2] )
    min_h = math.inf
    for el in shingles:
        hash_value = hash_f.get_hash_value(el)
        if hash_value < min_h:
            min_h = hash_value

    return (doc_id, min_h )

In [27]:
sig_matrix = docId_shingleset_hfunc.map(minhash_map).groupByKey().map(lambda x : (x[0], list(x[1])))

In [32]:
sig_matrix.count()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "spark-3.4.0-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "spark-3.4.0-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [28]:
def map_buckets(row):

    band_nr = 5
    row_nr = 10
    doc_id = row[0]
    doc_sign = row[1]
    hash_funct = hashFamily(1)
    out = []

    for i in range(0,band_nr):
        band_id = i
        idx = i*row_nr
        set_col = ' '.join(str(x) for x in doc_sign[idx:idx+row_nr])
        bucket = hash_funct.get_hash_value(set_col)
        out.append( ( (band_id, bucket), doc_id)  )

    return iter(out) #since we are going to return multiple tuple we call flatMap() and return an iterator on those tuple


In [29]:
candidates = sig_matrix.flatMap(map_buckets).groupByKey().map(lambda x : (x[0], list(x[1])))

In [30]:
similar_pairs = set()
sig_df = sig_matrix
sig_df.persist() #store it in memory to avoid lazy behaviour
sig_df = sig_df.collect() #collect all the distributed record to speedup computation
start_time = time.time()

for candidates_list in candidates.map(lambda x: x[1]).collect(): #collecting all candidate lists from worker nodes

    candidates_nr = len(candidates_list)

    for i in range(0,candidates_nr-1):
        for j in range(i+1,candidates_nr):

            doc_id_1 = candidates_list[i]
            doc_id_2 = candidates_list[j]

            docId_Sig_1 = sig_df[doc_id_1]
            docId_Sig_2 = sig_df[doc_id_2]
            if docId_Sig_1[0] != doc_id_1:
                raise Exception("DocId is %d while retrived %d from signature matrix."%(doc_id_1,docId_Sig_1[0]))
            if docId_Sig_2[0] != doc_id_2:
                raise Exception("DocId is %d while retrived %d from signature matrix."%(doc_id_1,docId_Sig_2[0]))

            sig_1 = set(docId_Sig_1[1]) #transform signature into a set
            sig_2 = set(docId_Sig_2[1])
            #sig_1 = sig_df.filter("_1 =="+" "+ str(doc_id_1) ).select('_2') #get signature of doc 1
            #sig_2 = sig_df.filter("_1 =="+" "+ str(doc_id_2) ).select('_2') #get signature of doc 2

            #sig_2 = set(sig_2.collect()[0]['_2'])
            js = len(sig_1.intersection(sig_2) ) / len(sig_1.union(sig_2) ) #Compute Jaccard'Similarity
            if js >= 0.008:
                pair = tuple(sorted((doc_id_1,doc_id_2) ))
                similar_pairs.add(  pair   )

end_time = time.time()

lsh_time = end_time - start_time

print("FOUND %d SIMILAR PAIRS" %len(similar_pairs))

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "spark-3.4.0-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "spark-3.4.0-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [ ]:
similar_pairs